<a href="https://colab.research.google.com/github/BruceXavierChou/stock_analysis/blob/main/%E3%80%8Cstk_ch07_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-07 年報問答機器人

## 7-2 取得年報

### 1️⃣  匯入套件

In [ ]:
import requests
from bs4 import BeautifulSoup

### 2️⃣ 建立函式-取得年報

In [ ]:
def annual_report(id,y):
  url = 'https://doc.twse.com.tw/server-java/t57sb01'
  # 建立 POST 請求的表單
  data = {
      "id":"",
      "key":"",
      "step":"1",
      "co_id":id,
      "year":y,
      "seamon":"",
      "mtype":'F',
      "dtype":'F04'
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data)
    # 取得回應後擷取檔案名稱
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a').text
    print(link1)
  except Exception as e:
    print(f"發生{e}錯誤")
  # 建立第二個 POST 請求的表單
  data2 = {
      'step':'9',
      'kind':'F',
      'co_id':id,
      'filename':link1 # 檔案名稱
  }
  try:
    # 發送 POST 請求
    response = requests.post(url, data=data2)
    link=BeautifulSoup(response.text, 'html.parser')
    link1=link.find('a')
    # 取得 PDF 連結
    link2 = link1.get('href')
    print(link2)
  except Exception as e:
    print(f"發生{e}錯誤")
  # 發送 GET 請求
  try:
    response = requests.get('https://doc.twse.com.tw' + link2)
    # 取得 PDF 資料
    with open(y + '_' + id + '.pdf', 'wb') as file:
        file.write(response.content)
    print('OK')
  except Exception as e:
    print(f"發生{e}錯誤")

### 3️⃣ 呼叫函式

In [ ]:
annual_report('2330','113')

2023_2330_20240604F04.pdf
/pdf/2023_2330_20240604F04_20240719_105100.pdf
OK


## 7-3 年報問答

###4️⃣ 安裝相關套件

由於版本更新的相容性問題, pdfplumber使用舊版本的0.10.2

In [ ]:
!pip install langchain==0.0.335 openai tiktoken pdfplumber==0.10.2 faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


###  5️⃣ 匯入相關套件

In [ ]:
import os
import getpass
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI

### 6️⃣ 設定環境變數和建立 OpenAI 模型

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
llm_16k = ChatOpenAI(model="gpt-3.5-turbo")

OpenAI API Key:··········


### 7️⃣ 建立函式-建立向量資料庫

In [ ]:
def pdf_loader(file,size,overlap):
  loader = PDFPlumberLoader(file)
  doc = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(
                          chunk_size=size,
                          chunk_overlap=overlap)
  new_doc = text_splitter.split_documents(doc)
  db = FAISS.from_documents(new_doc, OpenAIEmbeddings())
  return db

### 8️⃣  呼叫函式

In [ ]:
db = pdf_loader('/content/113_2330.pdf',500,50)

### 9️⃣ 查詢相關資料

In [ ]:
query = "公司是否有明確的成長或創新策略?"
docs = db.similarity_search(query,k=3)
for i in docs:
    print(i.page_content)
    print('_________')

使命
創新是我們的成長的泉源。我們追求的是全面，涵
蓋策略、行銷、管理、技術、製造等各方面的創
作為全球邏輯積體電路產業中，長期且值得信賴的
新。創新不僅僅是有新的想法，還需要執行力，做
Fab 10 技術及產能提供者。
出改變，否則只是空想，沒有益處。
企業核心價值
客戶信任
客戶是我們的夥伴，因此我們優先考慮客戶的需
誠信正直
AP3 求。我們視客戶的競爭力為台積公司的競爭力，而
這是我們最基本也是最重要的理念。我們說真話； 客戶的成功也是台積公司的成功。我們努力與客戶
我們不誇張、不作秀；對客戶我們不輕易承諾，一 建立深遠的夥伴關係，並成為客戶信賴且賴以成功
旦做出承諾，必定不計代價，全力以赴；對同業 的長期重要夥伴。
II 001
_________
和先進封裝解決方案方面強大的技術領先，我們能夠掌握更多的產業成長機會。
我們專注於公司的業務基本面，有目標地執行我們的全球製造足跡策略，以支持客戶成長與增加他們的信任。我
劉德音 魏哲家
們將繼續推動台積公司在全球所有晶圓廠的數位卓越化，並致力實現全方位的智慧化和自動化製造。無論我們在
董事長 總裁
何處營運，我們都決心要成為具有最高效率和最具成本效益的半導體製造者。
010 011
_________
前提下追求收益。一般而言，該政策要求須投資於投資
可能使台積公司必須採取以下措施：（一）購買、使用並 於獲得融資前削減、修改或是延遲產能擴充計劃。
144 145
_________


### 🔟  匯入問答相關套件

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA

### 1️⃣1️⃣  建立函式-問答程式

In [ ]:
# 提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "你是一個根據年報資料與上下文作回答的助手,"
     "如果有明確數據或技術(產品)名稱可以用數據或名稱回答,"
     "回答以繁體中文和台灣用語為主。"
     "{context}"),
    ("human","{question}")])

# 建立問答函式
def question_and_answer(question):
    qa = RetrievalQA.from_llm(llm=llm_16k,
                              prompt=prompt,
                              return_source_documents=True,
                              retriever=db.as_retriever(
                                  search_kwargs={'k':10}))
    result = qa(question)
    return result

### 1️⃣2️⃣ 建立迴圈進行問答

In [ ]:
while True:
    question = input("輸入問題:")
    if not question.strip():
        break
    result=question_and_answer(question)
    print(result['result'])
    print('_________')
    #print(result["source_documents"])

輸入問題:公司是否有明確的成長或創新策略?
是的，根據年報資料，台積公司有明確的成長和創新策略。他們強調創新是公司成長的泉源，追求全面的創新涵蓋策略、行銷、管理、技術、製造等各方面，並致力成為全球邏輯積體電路產業中長期且值得信賴的創新者。此外，台積公司也專注於公司的業務基本面，有目標地執行全球製造足跡策略，以支持客戶成長並增加他們的信任。他們致力於推動數位卓越化，在全球所有晶圓廠實現全方位的智慧化和自動化製造，以提供更多產業成長機會。
_________
輸入問題:公司目前正在開發的項目是?
公司目前正在開發的項目包括新的製程技術-N4X和N3X，提供給高效能運算產品使用。此外，公司也在推行健康促進活動和持續改善產品品質、生產效率以及降低生產成本。
_________
輸入問題:公司未來的展望?
台積公司的未來展望看好，他們專注於全球製造足跡策略，致力提供最有效率和成本效益的半導體製造服務。隨著AI技術持續發展，台積公司將扮演更重要的角色，因為AI模型需要更強大的半導體硬體支援。台積公司透過先進製程技術和封裝解決方案的技術領先地位，能夠把握更多產業成長機會。此外，台積公司也重視永續發展，積極推動ESG（環境、社會、公司治理）相關議題，努力打造更多元、永續的未來。整體而言，台積公司在技術領先、業務基本面和全球製造策略上持續進步，展望一片光明。
_________


KeyboardInterrupt: Interrupted by user

## 7-4 年報總結與分析

### 1️⃣3️⃣ 回答結果及原始資料

In [ ]:
from langchain.chains.summarize import load_summarize_chain

### 1️⃣4️⃣ 總結原始資料

In [ ]:
# 建立關鍵字串列
key_word = ['有關市場策略的調整或變化有何提及？',
          '公司對未來一年的展望是什麼？',
          '公司的總收入是否增長，淨利潤的正負情況是否有變化？',
          '國際競爭及海外市場情況',
          '目前的研發狀況?']
data_list = []
for word in key_word:
    data = db.max_marginal_relevance_search(word)
    # 整合 Document 串列
    data_list += data

# 建立提示訊息串列
prompt_template = [("system","你的任務是生成年報摘要。"
                "我們提供年報{text}請你負責生成,"
                "且要保留重點如營收漲跌、開發項目等,"
                "最後請使用繁體中文輸出報告")]
prompt = ChatPromptTemplate.from_messages(messages=prompt_template)

### 1️⃣5️⃣  呼叫函式

In [ ]:
chain_refine_16k = load_summarize_chain(llm=llm_16k,
                                        chain_type='stuff',
                                        prompt=prompt)
print(chain_refine_16k.run(data_list))

根據台積公司的年報，今年度營收持續增長，營業淨利也有顯著提升。公司在開發先進製程技術和封裝解決方案方面取得重大進展，致力於滿足客戶需求並提升競爭力。此外，公司在環境保護和氣候變遷管理方面也積極採取措施，並持續投入研發項目，包括5G和智能物聯網應用等新技術。台積公司在全球半導體市場佔有重要地位，並致力於實現永續發展目標，朝向淨零排放願景邁進。


### 1️⃣6️⃣  提取關鍵字

In [ ]:
from langchain.chains import LLMChain
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

word_prompt=ChatPromptTemplate.from_messages(messages=[
    ("human","從{input}聯想出4個與年報分析有關的重要關鍵字,"\
     "請確保回答具有具有關聯性、多樣性和變化性。 \n "
     "僅回覆關鍵字, 並以半形逗號與空格來分隔。不要加入其他內容"
    "")]
)
word_chain = LLMChain(llm=llm_16k, prompt=word_prompt)
output_parser.parse(word_chain('公司的營收狀況如何？')['text'])

['增長', '利潤率', '現金流量', '市佔率']

### 1️⃣7️⃣ 設定 AI 角色讓其分析報告

In [ ]:
data_prompt=ChatPromptTemplate.from_messages(messages=[
    ("system","你現在是一位專業的股票分析師,"
    "你會以詳細、嚴謹的角度進行年報分析, 針對{output}作分析並提及重要數字\
    ,然後生成一份專業的趨勢分析報告。"),
    ("human","{text}")])
data_chain = LLMChain(llm=llm_16k, prompt=data_prompt)

### 1️⃣8️⃣ 整合函式

In [ ]:
def analyze_chain(input):
    # 搜尋「問題」的相關資料
    data = db.max_marginal_relevance_search(input)

    # 第一個 Chain 元件, 建立「關鍵字」串列
    word = word_chain(input)
    word_list = output_parser.parse(word['text'])

    # 搜尋「關鍵字」的相關資料
    for i in word_list:
      data += db.max_marginal_relevance_search(i,k=2)
    word_list.append(input)

    # 第二個 Chain 元件, 生成分析報告
    result = data_chain({'output':word_list,'text':data})

    return result['text']

### 1️⃣9️⃣ 呼叫函式

In [ ]:
input = '公司的營收狀況如何？'
print(analyze_chain(input))

根據提供的年報資料，我們來進行分析：

1. **成長**:
   - 本年度綜合損益總額減少，主要是因為淨利下降所致。
   - 其他營業收益淨額增加，主要是因為認列處分及報廢不動產、廠房及設備淨益。

2. **利潤率**:
   - 本年度淨利下降，導致歸屬予母公司業主綜合損益下降。
   - 現金流量比率下降，主要是因為營業活動淨現金流量減少。

3. **市佔率**:
   - 營業外收入及支出增加，主要是因利息收入及子公司損益份額增加。
   - 總資產週轉率減少，可能表示公司的資產使用效率下降。

4. **現金流量**:
   - 應收款項週轉率下降，可能表示公司在收款方面遇到了困難。
   - 現金再投資比率下降，可能表示公司在再投資方面有所減少。

5. **公司的營收狀況如何？**:
   - 本年度綜合損益總額減少，主要是因淨利下降所致。
   - 公司的利潤率、現金流量比率、資產報酬率等指標均呈現下降趨勢。

綜合以上分析，公司在本年度面臨一些挑戰，如淨利下降、資產使用效率下降等，建議公司應加強盈利能力，提高資產運用效率，並謹慎管理現金流量。未來需密切關注公司的營運狀況，以制定適當的因應措施。
